# MySQL-Datenbank "la mediterranea" 

## Import des Python Moduls, Herstellung der Verbindung

## MySQL-Datenbank

<https://v078803.kasserver.com/mysqladmin/>  
Benutzername: v078803  
Passwort: HibaIrinaPuyaHeiko2024_  
Datenbankname: v078803


In [ ]:
import mysql.connector
from mysql.connector import Error

### lokale Datenbank
config = {
    'user': 'v078803',
    'password': 'HibaIrinaPuyaHeiko2024_',
    'host': 'localhost',
    'unix_socket': '/Applications/MAMP/tmp/mysql/mysql.sock',
    'database': 'v078803',
    'raise_on_warnings': True
}


### Server Datenbank
""" config = {
    'user': 'v078803',
    'password': 'HibaIrinaPuyaHeiko2024_',
    'host': 'v078803.kasserver.com',
    'database': 'v078803',
    'raise_on_warnings': True
} """

# connection zum MySQL-Server herstellen
db_connection = mysql.connector.connect(**config)

print("Verbunden mit:", db_connection.get_server_info())


### Erstellen der Tabelle playlist

||__playlist__|
|---|---|
|PK|track_id|
||track_artist|
||track_bpm|
||track_device|
||track_label|
||track_timestamp|
|__FK__|rekordbox_id|

In [ ]:
try:
    # connection zum MySQL-Server herstellen
    db_connection = mysql.connector.connect(**config)
    if db_connection.is_connected():
        print("Verbunden mit:", db_connection.get_server_info())
        print("Erstelle die Tabellen...")
        db_cursor = db_connection.cursor()
        ### Tabelle 'conditions' erstellen
        sql_query = """
            CREATE TABLE playlist (
                track_id INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
                track_artist VARCHAR(255),
                track_bpm FLOAT,
                track_device VARCHAR(255),
                track_genre VARCHAR(255),
                track_key VARCHAR(255),
                track_label VARCHAR(255),
                track_title VARCHAR(255),
                track_timestamp TIMESTAMP,
                rekordbox_id INT NOT NULL
            )
        """
        db_cursor.execute(sql_query)
        print("Tabelle 'playlist' erfolgreich erstellt.")
    else:
        print("Verbindung konnte nicht hergestellt werden.")
except Error as e:
    print("Fehler beim Verbinden mit MySQL", e)
finally:
    if (db_connection.is_connected()):
        db_cursor.close()
        db_connection.close()
        print("MySQL-Verbindung ist geschlossen.")

### Playlisten vom Tracked Update Expression Trigger von BLT

In [2]:
import socket
import threading
import json
import mysql.connector
from mysql.connector import Error

# Globale Variablen für Track-Daten und Track-Historie
track_data = {}
track_history = {}

# Placeholder für Socket.IO (hier muss die tatsächliche Socket.IO-Initialisierung erfolgen)
class Socket:
    def emit(self, event, data):
        print(f"Event: {event}, Data: {data}")

socketio = Socket()

def convert_time(milliseconds):
    """Konvertiert Millisekunden in ein Minuten:Sekunden-Format."""
    seconds = milliseconds / 1000
    minutes = int(seconds // 60)
    seconds = int(seconds % 60)
    return f"{minutes:02}:{seconds:02}"

def update_track_info(json_data):
    """Aktualisiert die Track-Informationen, sendet sie über Socket.IO und speichert sie in der MySQL-Datenbank und in der Historie."""
    global track_data, track_history
    track_data = {
        "track_artist": json_data.get("artist"),
        "track_bpm": round(json_data.get("bpm"), 2) if json_data.get("bpm") is not None else None,
        "track_device": json_data.get("device"),
        "track_genre": json_data.get("track_genre"),
        "track_key": json_data.get("key"),
        "track_label": json_data.get("label"),
        "track_timestamp": json_data.get("timestamp"),
        "track_title": json_data.get("title"),
        "rekordbox_id": json_data.get("id")
    }
    socketio.emit('update', track_data)
    save_to_database(track_data)
    add_to_history(track_data)

def save_to_database(track_data):
    """Speichert die Track-Daten in der MySQL-Datenbank."""
    try:
        db_connection = mysql.connector.connect(**config)
        
        if db_connection.is_connected():
            cursor = db_connection.cursor()
            insert_query = """
                INSERT INTO playlist (track_artist, track_bpm, track_device, track_genre, track_key, track_label, track_timestamp, track_title, rekordbox_id)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
            track_data_tuple = (
                track_data["track_artist"],
                track_data["track_bpm"],
                track_data["track_device"],
                track_data["track_genre"],
                track_data["track_key"],
                track_data["track_label"],
                track_data["track_timestamp"],
                track_data["track_title"],
                track_data["rekordbox_id"]
            )
            cursor.execute(insert_query, track_data_tuple)
            db_connection.commit()
            print("Track-Daten erfolgreich in die Datenbank geschrieben.")
    except Error as e:
        print("Fehler beim Verbinden mit der MySQL-Datenbank", e)
    finally:
        if (db_connection.is_connected()):
            cursor.close()
            db_connection.close()
            print("MySQL-Verbindung geschlossen.")

def add_to_history(track_data):
    """Fügt die Track-Daten der Track-Historie hinzu."""
    global track_history
    track_id = track_data["rekordbox_id"]
    track_history[track_id] = track_data
    print("Track-Daten erfolgreich zur Historie hinzugefügt:", track_history)

def socket_receiver():
    """Empfängt UDP-Pakete und verarbeitet die JSON-Daten."""
    # Erstellen eines UDP-Sockets
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    port = 7001
    sock.bind(('', port))
    print("Socket verbunden mit Port", port)
    receiving = True

    try:
        while receiving:
            print('\nWarten auf eine Nachricht')
            data, address = sock.recvfrom(4096)
            print('Erhaltene {} Bytes von {}'.format(len(data), address))

            try:
                json_data = json.loads(data.decode('utf-8'))  # Daten dekodieren
                print("JSON-Daten erfolgreich dekodiert:", json_data)
                update_track_info(json_data)
            except json.JSONDecodeError as e:
                print("Fehler beim Dekodieren der JSON-Daten:", e)
    finally:
        sock.close()  # Socket schließen

if __name__ == '__main__':
    receiver_thread = threading.Thread(target=socket_receiver)
    receiver_thread.start()

Socket verbunden mit Port 7001

Warten auf eine Nachricht


Erhaltene 148 Bytes von ('127.0.0.1', 52757)
JSON-Daten erfolgreich dekodiert: {'timestamp': '2024-07-21T11:57:17.139235', 'device': 'XDJ-XZ', 'artist': 'Depeche Mode', 'id': 7737, 'title': 'Broken', 'label': 'Columbia/Mute', 'bpm': 119.99}
Event: update, Data: {'track_artist': 'Depeche Mode', 'track_bpm': 119.99, 'track_device': 'XDJ-XZ', 'track_genre': None, 'track_key': None, 'track_label': 'Columbia/Mute', 'track_timestamp': '2024-07-21T11:57:17.139235', 'track_title': 'Broken', 'rekordbox_id': 7737}
Track-Daten erfolgreich in die Datenbank geschrieben.
MySQL-Verbindung geschlossen.
Track-Daten erfolgreich zur Historie hinzugefügt: {7737: {'track_artist': 'Depeche Mode', 'track_bpm': 119.99, 'track_device': 'XDJ-XZ', 'track_genre': None, 'track_key': None, 'track_label': 'Columbia/Mute', 'track_timestamp': '2024-07-21T11:57:17.139235', 'track_title': 'Broken', 'rekordbox_id': 7737}}

Warten auf eine Nachricht
Erhaltene 143 Bytes von ('127.0.0.1', 56709)
JSON-Daten erfolgreich dekod